In [1]:
import cv2
from keras.models import  model_from_json

json_file=open('emotion_model.json','r')
loaded_model=json_file.read()
json_file.close()
model=model_from_json(loaded_model)

2023-07-13 09:40:27.290934: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-13 09:40:27.535312: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-13 09:40:27.538804: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 09:40:29.509913: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-13 09:40:32.929891: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [2]:
model.load_weights('emotion_model.h5')

In [3]:
import numpy as np
import gradio as gr

emotion_label={0:"angry",1:"disgusted",2:"fearful",3:"happy",4:"neutral",5:"sad",6:"surprised"}

cap=cv2.VideoCapture(-1)
while True:
    result,img=cap.read()
    if not result:
        break
    img=cv2.resize(img,(800,800))

    face_detector=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_alt2.xml')
    gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    num_faces=face_detector.detectMultiScale(gray_img,scaleFactor=1.3,minNeighbors=5)
    
    for(x,y,w,h) in num_faces:
        cv2.rectangle(img,(x,y-50),(x+w,y+h+10),(0,255,0),4)
        roi_gray_img=gray_img[y:y+h,x:x+w]
        cropped_img=np.expand_dims(np.expand_dims(cv2.resize(roi_gray_img,(48,48)),-1),0)

        emotion_prediction=model.predict(cropped_img)
        maxindex=int(np.argmax(emotion_prediction))
        cv2.putText(img,emotion_label[maxindex],(x+5,y-20),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2,cv2.LINE_AA)
    
    cv2.imshow('video',img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
    



/home/rdev/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/rdev/.local/lib/python3.10/site-packages/cv2/qt/plugins"


1/1 [==============================] - 0s 37ms/step


In [22]:
from flask import Flask, render_template, Response
import cv2

app = Flask('hello')
cap = cv2.VideoCapture(-1) 

def gen_frames():  
    while True:
        result,img=cap.read()

        img=cv2.resize(img,(800,800))
        if not result:
            break
        face_detector=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_alt2.xml')
        gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        num_faces=face_detector.detectMultiScale(gray_img,scaleFactor=1.3,minNeighbors=5)
        
        for(x,y,w,h) in num_faces:
            cv2.rectangle(img,(x,y-50),(x+w,y+h+10),(0,255,0),4)
            roi_gray_img=gray_img[y:y+h,x:x+w]
            cropped_img=np.expand_dims(np.expand_dims(cv2.resize(roi_gray_img,(48,48)),-1),0)

            emotion_prediction=model.predict(cropped_img)
            maxindex=int(np.argmax(emotion_prediction))
            cv2.putText(img,emotion_label[maxindex],(x+5,y-20),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        # success, frame = camera.read()
        # if not success:
        #     break
        # else:

        ret, buffer = cv2.imencode('.jpg', img)
        img = buffer.tobytes()
        yield (b'--frame\r\nContent-Type: image/jpeg\r\n\r\n' + img + b'\r\n')

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return """
    <html>
    <head>
    </head>
    <body style="margin:0">
    <div class="container">
        <div class="row">
            <div class="col-lg-8  offset-lg-2" style="text-align:center;">
                <h3 class="mt-5" style="background-color:#82EEFD;font-size:30px;padding:10px;text-decoration:underline;">Emotion Detection</h3>
                <img src="/video_feed" width="800px" height="800px" style="border:5px solid black">
            </div>
        </div>
    </div>
    </body> 
    </html>       
    """
app.run()

 * Serving Flask app 'hello'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Jul/2023 21:21:15] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 185ms/step


127.0.0.1 - - [12/Jul/2023 21:21:16] "GET /video_feed HTTP/1.1" 200 -


1/1 [==============================] - 0s 32ms/step


In [23]:
cap.release()

1/1 [==============================] - 0s 36ms/step


Error on request:
Traceback (most recent call last):
  File "/home/rdev/.local/lib/python3.10/site-packages/werkzeug/serving.py", line 364, in run_wsgi
    execute(self.server.app)
  File "/home/rdev/.local/lib/python3.10/site-packages/werkzeug/serving.py", line 327, in execute
    for data in application_iter:
  File "/home/rdev/.local/lib/python3.10/site-packages/werkzeug/wsgi.py", line 289, in __next__
    return self._next()
  File "/home/rdev/.local/lib/python3.10/site-packages/werkzeug/wrappers/response.py", line 32, in _iter_encoded
    for item in iterable:
  File "/tmp/ipykernel_21797/3350768940.py", line 11, in gen_frames
    img=cv2.resize(img,(800,800))
cv2.error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
